# Verification: CRPSS 

In [2]:
import os
import sys
import time
import numpy as np
from glob import glob
from datetime import datetime, timedelta
from scipy.interpolate import RegularGridInterpolator

import h5py

In [3]:
#import numba as nb

In [4]:
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/')
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/libs/')

from namelist import *
import data_utils as du
import verif_utils as vu

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
with h5py.File(save_dir+'CNN_domain.hdf', 'r') as h5io:
    lon_GFS = h5io['lon_GFS'][...]
    lat_GFS = h5io['lat_GFS'][...]

In [7]:
#name_output = '/glade/campaign/cisl/aiml/ksha/LDM_results/LDM_2023_ini00_lead{:02d}.hdf'

**CRPS climatology reference in 2023**

In [8]:
year = 2023
N_days = 300
name_MRMS = '/glade/campaign/cisl/aiml/ksha/GFS/MRMS_01H_y{}_025.hdf'.format(year)

In [9]:
with h5py.File(name_MRMS, 'r') as h5io:
    MRMS = h5io['MRMS'][...]
N_total = len(MRMS)

## 0.25 deg 3H results

In [14]:
with h5py.File(save_dir+'ERA5_CDFs.hdf', 'r') as h5io:
    ERA5_CDFs = h5io['ERA5_CDFs'][...]
    indx_in_GFS = h5io['indx_in_GFS'][...]
    indy_in_GFS = h5io['indy_in_GFS'][...]
    
ERA5_CDFs = ERA5_CDFs # mm/hour to mm/3-hour

q_bins = np.arange(0, 1, 0.01)
N_bins = len(q_bins)
grid_shape = lon_GFS.shape # MRMS grid shape

MRMS_lead = np.empty((N_days, grid_shape[0], grid_shape[1]))

LEADs = [3,] #np.arange(3, 168+3, 3)
CRPS_climo_lead = np.empty((N_days,)+grid_shape); 

for l, lead in enumerate(LEADs):

    name_check = '/glade/work/ksha/GAN_result/CRPS_MRMS_3H_lead{:02d}_Climo_2023.hdf'.format(lead)
    
    if os.path.isfile(name_check) is True:
        # re-fill NaN for a new lead time
        CRPS_climo_lead[...] = np.nan
        
        # collect MRMS on the forecasted time
        MRMS_lead[...] = np.nan

        # hourly to 3-hourly
        for d in range(N_days):
            h1 = d*24 + lead
            h0 = h1-3
            MRMS_lead[d, ...] = np.sum(MRMS[h0:h1, ...], axis=0)/3

        # compute CRPS on each grid cell
        for ix in range(grid_shape[0]):
            for iy in range(grid_shape[1]):
                MRMS_TRUE_ = MRMS_lead[:, ix, iy][..., None]
                CDFs = ERA5_CDFs[:, ix, iy][..., None]
                CRPS_climo_lead[:, ix, iy] = vu.CRPS_1d_from_quantiles(q_bins, CDFs, MRMS_TRUE_)[:, 0]
                
        CRPS_climo_lead[np.isnan(MRMS_lead)] = np.nan
        
        # # backup every lead time  
        # tuple_save = (CRPS_climo_lead,)
        # label_save = ['CRPS_climo_lead',]
        # du.save_hdf5(tuple_save, label_save, result_dir, 'CRPS_MRMS_3H_lead{:02d}_Climo_2023.hdf'.format(lead))

In [10]:
crps_climo_name = '/glade/work/ksha/GAN_result/CRPS_MRMS_3H_lead{:02d}_Climo_2023.hdf'

with h5py.File(crps_climo_name.format(3), 'r') as h5io:
    crps_ = h5io['CRPS_climo_lead'][...]

In [11]:
np.nanmean(crps_)

0.17808720673904035

In [15]:
np.nanmean(CRPS_climo_lead)

0.0593624022463468

## 0.25 deg 6H results

In [13]:
with h5py.File(save_dir+'ERA5_CDFs.hdf', 'r') as h5io:
    ERA5_CDFs = h5io['ERA5_CDFs'][...]
    indx_in_GFS = h5io['indx_in_GFS'][...]
    indy_in_GFS = h5io['indy_in_GFS'][...]
    
ERA5_CDFs = 6*ERA5_CDFs # mm/hour to mm/3-hour

q_bins = np.arange(0, 1, 0.01)
N_bins = len(q_bins)
grid_shape = lon_GFS.shape # MRMS grid shape

MRMS_lead = np.empty((N_days, grid_shape[0], grid_shape[1]))

LEADs = [6,] #np.arange(6, 168+6, 6)
CRPS_climo_lead = np.empty((N_days,)+grid_shape); 

for l, lead in enumerate(LEADs):

    name_check = '/glade/work/ksha/GAN_result/CRPS_MRMS_6H_025_lead{:02d}_Climo_2023.hdf'.format(lead)
    
    if os.path.isfile(name_check) is False:
        # re-fill NaN for a new lead time
        CRPS_climo_lead[...] = np.nan
        
        # collect MRMS on the forecasted time
        MRMS_lead[...] = np.nan

        # hourly to 6-hourly
        for d in range(N_days):
            h1 = d*24 + lead
            h0 = h1-6
            MRMS_lead[d, ...] = np.sum(MRMS[h0:h1, ...], axis=0)

        # compute CRPS on each grid cell
        for ix in range(grid_shape[0]):
            for iy in range(grid_shape[1]):
                MRMS_TRUE_ = MRMS_lead[:, ix, iy][..., None]
                CDFs = ERA5_CDFs[:, ix, iy][..., None]
                CRPS_climo_lead[:, ix, iy] = vu.CRPS_1d_from_quantiles(q_bins, CDFs, MRMS_TRUE_)[:, 0]
                
        CRPS_climo_lead[np.isnan(MRMS_lead)] = np.nan
        
        # backup every lead time  
        # tuple_save = (CRPS_climo_lead,)
        # label_save = ['CRPS_climo_lead',]
        # du.save_hdf5(tuple_save, label_save, result_dir, 'CRPS_MRMS_6H_lead{:02d}_Climo_2023.hdf'.format(lead))

In [14]:
np.mean(CRPS_climo_lead)

nan

In [15]:
np.nanmean(CRPS_climo_lead)

0.374969892359708

## 0.1 deg 6H results

In [ ]:
# with h5py.File(save_dir+'ERA5_CDFs.hdf', 'r') as h5io:
#     ERA5_CDFs = h5io['ERA5_CDFs'][...]
#     indx_in_GFS = h5io['indx_in_GFS'][...]
#     indy_in_GFS = h5io['indy_in_GFS'][...]
# ERA5_CDFs = 6*ERA5_CDFs # mm/hour to mm/6-hour

# q_bins = np.arange(0, 1, 0.01)
# N_bins = len(q_bins)
# grid_shape = lon_01.shape # MRMS grid shape

# MRMS_lead = np.empty((N_days, grid_shape[0], grid_shape[1]))

# LEADs = np.arange(6, 168+6, 6)
# CRPS_climo_lead = np.empty((N_days,)+grid_shape); CRPS_climo_lead[...] = np.nan

# for l, lead in enumerate(LEADs):
    
#     name_check = '/glade/work/ksha/GAN_result/CRPS_MRMS_6H_lead{:02d}_Climo_2023.hdf'.format(lead)
    
#     if os.path.isfile(name_check) is False:
#         # collect MRMS on the forecasted time
#         MRMS_lead[...] = np.nan
#         for d in range(N_days):
#             h1 = d*24 + lead
#             h0 = h1-6
#             MRMS_lead[d, ...] = np.sum(MRMS[h0:h1, ...], axis=0)
            
#         for ix in range(grid_shape[0]):
#             for iy in range(grid_shape[1]):
#                 MRMS_TRUE_ = MRMS_lead[:, ix, iy][..., None]
#                 CDFs = ERA5_CDFs[:, indx_in_GFS[ix, iy], indy_in_GFS[ix, iy]][..., None]
#                 CRPS_climo_lead[:, ix, iy] = vu.CRPS_1d_from_quantiles(q_bins, CDFs, MRMS_TRUE_)[:, 0]
    
#         # backup every lead time  
#         tuple_save = (CRPS_climo_lead,)
#         label_save = ['CRPS_climo_lead',]
#         du.save_hdf5(tuple_save, label_save, result_dir, 'CRPS_MRMS_6H_lead{:02d}_Climo_2023.hdf'.format(lead))

In [21]:
# np.mean(CRPS_climo_lead)

0.5644630928538301

**Compare to 3-h CRPS reference**

In [ ]:
# q_bins = np.arange(0, 1, 0.01)
# grid_shape = lon_01.shape # MRMS grid shape

# LEADs = np.arange(3, 168+3, 3)
# CRPS_climo = np.empty((N_days, len(LEADs))+grid_shape); CRPS_climo[...] = np.nan

# for l, lead in enumerate(LEADs):

#     for day in range(N_days):
#     with h5py.File(name_output.format(lead)) as h5io:
#         MRMS_TRUE = h5io['MRMS_TRUE'][...]
#         MRMS_PRED = h5io['MRMS_PRED'][...]
    
#     for ix in range(grid_shape[0]):
#         for iy in range(grid_shape[1]):
#             MRMS_TRUE_ = MRMS_TRUE[:, ix, iy][..., None]
#             CDFs = ERA5_CDFs[:, indx_in_GFS[ix, iy], indy_in_GFS[ix, iy]][..., None]
#             CRPS_climo[:, l, ix, iy] = vu.CRPS_1d_from_quantiles(q_bins, CDFs, MRMS_TRUE_)[:, 0] # size=(100, 1)
            
#     # backup every lead time  
#     tuple_save = (CRPS_climo,)
#     label_save = ['CRPS_climo',]
#     du.save_hdf5(tuple_save, label_save, result_dir, 'CRPS_MRMS_climo_2023.hdf')

In [19]:
lead = 6
with h5py.File(result_dir+'CRPS_MRMS_6H_lead{:02d}_Climo_2023.hdf'.format(lead), 'r') as h5io:
    CRPS_climo_6h = h5io['CRPS_climo_lead'][...]

**CRPS from LDF outputs**

In [ ]:
# N_time = 300
# grid_shape = lon_01.shape # MRMS grid shape
# LEADs = [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36]
# CRPS_PRED = np.empty((N_time, len(LEADs))+grid_shape); CRPS_PRED[...] = np.nan

# for l, lead in enumerate(LEADs):

#     with h5py.File(name_output.format(lead)) as h5io:
#         MRMS_TRUE = h5io['MRMS_TRUE'][...]
#         MRMS_PRED = h5io['MRMS_PRED'][...]

#     MRMS_TRUE_flat = MRMS_TRUE.reshape(300, 256*576)
#     MRMS_PRED_flat = MRMS_PRED.reshape(300, 10, 256*576)

#     # handle NaNs
#     for i in range(300):
#         for k in range(256*576):
#             if (np.sum(np.isnan(MRMS_PRED_flat[i, :, k])) > 0) and (np.isnan(MRMS_TRUE_flat[i, k]) is False):
#                 MRMS_PRED_flat[i, :, k] = 0

#     CRPS_pred, _, _ = vu.CRPS_1d_nan(MRMS_TRUE_flat, MRMS_PRED_flat)
#     CRPS_PRED[:, l, ...] = CRPS_pred.reshape((300, 256, 576))
    
#     # backup every lead time  
#     tuple_save = (CRPS_PRED,)
#     label_save = ['CRPS_PRED',]
#     du.save_hdf5(tuple_save, label_save, result_dir, 'CRPS_MRMS_LDM_2023.hdf')